# Initial experiments on bigger datasets

Same idea as the main notebook, but with different datesets:

* M1 species with minimum images per species ranging from 2 to 4
* Top ten genera

Conclusions:

* With top-3 accuracy of ~50% on the `rls-species-min-images-2`, it's viable for interactive tagging. Given that the experiment uses a 50-50 validation split, it means that some species are tagged based on a single example. 
* The top-10 genera is another story with perfect top-3 accuracy. It isn't practically interesting, though, as these genera are pretty distinct. But it may be useful to show likely genera alongside likely species.
* For the app use case, fitting on the full dataset without a validation set makes sense given the small amount of data. It doesn't appear too risky since the models don't appear to overfit too much with 120 epochs.

In [1]:
from fastai.vision.all import *
import mlflow

from ichthywhat import experiments

In [2]:
mlflow.set_tracking_uri("sqlite:////home/yanir/projects/deep-fish/mlruns.db")
mlflow.set_registry_uri("file:///home/yanir/projects/deep-fish/mlruns")

## Experiments from before the duplication bug fix

* Prefixed with `bad-` after the fact.

In [4]:
mlflow.set_experiment("bad-rls-species-min-images-4")
with mlflow.start_run(run_name="initial"):
    learner = experiments.create_reproducible_learner(
        resnet18, Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-4")
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    learner.fine_tune(120)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

epoch,train_loss,valid_loss,accuracy,top_3_accuracy,time
0,8.137529,6.529367,0.016949,0.036506,00:23


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,time
0,6.755535,6.331712,0.026076,0.048240,00:29
1,6.415333,6.127299,0.035202,0.065189,00:32
2,6.080937,5.930771,0.043025,0.080834,00:34
3,5.767301,5.726932,0.056063,0.116037,00:34
4,5.371635,5.533617,0.071708,0.138201,00:34
5,5.048445,5.357936,0.073012,0.153846,00:34
6,4.703758,5.195693,0.091265,0.174707,00:34
7,4.331200,4.999745,0.118644,0.212516,00:35
8,3.988313,4.840925,0.132986,0.250326,00:34
9,3.651376,4.664551,0.165580,0.276402,00:35


{'train_loss': 0.005885654129087925, 'valid_loss': 3.2181355953216553, 'accuracy': 0.4550195634365082, 'top_3_accuracy': 0.591916561126709, 'tta_accuracy': 0.49804434180259705, 'tta_top_3_accuracy': 0.6140808463096619}


In [3]:
mlflow.set_experiment("bad-rls-species-min-images-3")
with mlflow.start_run(run_name="initial"):
    class_names = [
        " ".join(path.name.split("-")[:2]).capitalize()
        for path in Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-3").ls()
    ]
    learner = experiments.create_reproducible_learner(
        resnet18,
        Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-3"),
        db_kwargs=dict(splitter=TrainTestSplitter(test_size=0.25, stratify=class_names, random_state=19)),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("splitter", "TrainTestSplitter(test_size=0.25, stratify=class_names, random_state=19)")
    learner.fine_tune(120)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

epoch,train_loss,valid_loss,accuracy,top_3_accuracy,time
0,8.677940,6.940779,0.017517,0.035796,00:29


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,time
0,7.254880,6.727590,0.022848,0.044935,00:40
1,6.892451,6.493554,0.033511,0.071592,00:43
2,6.559124,6.267745,0.051790,0.095202,00:46
3,6.199134,6.045044,0.068545,0.112719,00:46
4,5.803807,5.835220,0.082254,0.139375,00:46
5,5.459333,5.625960,0.095963,0.160701,00:46
6,5.097402,5.419718,0.115765,0.181264,00:45
7,4.796347,5.201528,0.131759,0.217060,00:46
8,4.442299,4.998625,0.145468,0.253618,00:46
9,4.051808,4.819652,0.164509,0.281797,00:46


{'train_loss': 0.006577446591109037, 'valid_loss': 3.0829033851623535, 'accuracy': 0.4729626774787903, 'top_3_accuracy': 0.6298552751541138, 'tta_accuracy': 0.5072353482246399, 'tta_top_3_accuracy': 0.6542269587516785}


In [4]:
mlflow.set_experiment("bad-rls-species-min-images-2")
with mlflow.start_run(run_name="initial"):
    class_names = [
        " ".join(path.name.split("-")[:2]).capitalize()
        for path in Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2").ls()
    ]
    learner = experiments.create_reproducible_learner(
        resnet18,
        Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2"),
        db_kwargs=dict(splitter=TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("splitter", "TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)")
    learner.fine_tune(120)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

INFO: 'initial-rls-species-min-images-2' does not exist. Creating a new experiment


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,time
0,9.105062,7.508452,0.007079,0.015766,00:37


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,time
0,7.929155,7.351262,0.011905,0.019627,00:42
1,7.621300,7.194269,0.013835,0.030888,00:42
2,7.317668,7.025113,0.021236,0.042149,00:43
3,7.050474,6.872590,0.030566,0.054698,00:42
4,6.686460,6.703837,0.036036,0.069498,00:42
5,6.393193,6.539725,0.045689,0.085586,00:43
6,6.099227,6.372160,0.057593,0.105856,00:43
7,5.788876,6.207588,0.071429,0.127735,00:43
8,5.401823,6.046915,0.083655,0.145109,00:42
9,5.065057,5.880597,0.100386,0.165701,00:43


{'train_loss': 0.004024412017315626, 'valid_loss': 4.281888008117676, 'accuracy': 0.3198198080062866, 'top_3_accuracy': 0.44530245661735535, 'tta_accuracy': 0.36776062846183777, 'tta_top_3_accuracy': 0.49581724405288696}


In [4]:
mlflow.set_experiment("bad-rls-species-min-images-2")
with mlflow.start_run(run_name="longer-freeze"):
    class_names = [
        experiments.get_species_from_path(path)
        for path in Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2").ls()
    ]
    print(f"{len(class_names)} images, {len(set(class_names))} classes")
    learner = experiments.create_reproducible_learner(
        resnet18,
        Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2"),
        db_kwargs=dict(splitter=TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("splitter", "TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)")
    mlflow.log_param("frozen_epochs", 20)
    mlflow.log_param("unfrozen_epochs", 40)
    learner.fine_tune(40, freeze_epochs=20)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

6215 images, 1696 classes


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,9.341114,8.117357,0.000965,0.002896,0.006435,00:30
1,9.242217,8.037866,0.001287,0.003861,0.009331,00:33
2,9.087565,7.919849,0.001609,0.005148,0.010618,00:34
3,8.811802,7.743546,0.003861,0.007079,0.016088,00:35
4,8.531291,7.540089,0.007722,0.012548,0.026384,00:36
5,8.069049,7.274133,0.010940,0.021236,0.049871,00:35
6,7.600770,6.970658,0.018662,0.037323,0.079151,00:36
7,7.058276,6.647336,0.035714,0.067568,0.126770,00:35
8,6.459054,6.328313,0.054698,0.099099,0.186615,00:35
9,5.750478,6.008142,0.073359,0.141892,0.239060,00:37


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,0.730613,4.511688,0.216860,0.349099,0.509652,00:44
1,0.654669,4.452454,0.225225,0.353282,0.509974,00:43
2,0.590169,4.424951,0.229086,0.362613,0.512227,00:43
3,0.522003,4.403265,0.228443,0.368726,0.521557,00:44
4,0.461296,4.363619,0.235521,0.377735,0.535393,00:45
5,0.436559,4.356511,0.238095,0.379344,0.536036,00:45
6,0.400406,4.379362,0.241634,0.379987,0.532497,00:44
7,0.385906,4.388885,0.228121,0.376126,0.531532,00:46
8,0.361071,4.430465,0.237452,0.367761,0.528314,00:44
9,0.346428,4.408989,0.234878,0.375804,0.537001,00:45


{'train_loss': 0.02981598488986492, 'valid_loss': 4.282947540283203, 'accuracy': 0.29279279708862305, 'top_3_accuracy': 0.4295366704463959, 'top_10_accuracy': 0.5736808180809021, 'tta_accuracy': 0.3452380895614624, 'tta_top_3_accuracy': 0.4794079661369324, 'tta_top_10_accuracy': 0.6190476417541504}


In [5]:
mlflow.set_experiment("bad-rls-species-min-images-2")
with mlflow.start_run(run_name="shorter-freeze"):
    class_names = [
        experiments.get_species_from_path(path)
        for path in Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2").ls()
    ]
    print(f"{len(class_names)} images, {len(set(class_names))} classes")
    learner = experiments.create_reproducible_learner(
        resnet18,
        Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2"),
        db_kwargs=dict(splitter=TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("splitter", "TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)")
    mlflow.log_param("frozen_epochs", 5)
    mlflow.log_param("unfrozen_epochs", 120)
    learner.fine_tune(120, freeze_epochs=5)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

6215 images, 1696 classes


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,9.327343,8.070066,0.000965,0.003861,0.007400,00:30
1,9.062040,7.712436,0.002252,0.007079,0.018983,00:32
2,8.416906,7.100293,0.017375,0.037001,0.064994,00:33
3,7.430519,6.440507,0.046332,0.088160,0.167632,00:34
4,6.391944,5.907843,0.080116,0.149614,0.257400,00:35


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,4.537655,5.869329,0.087838,0.162162,0.276384,00:41
1,4.427400,5.783800,0.095882,0.174710,0.286680,00:42
2,4.264870,5.681686,0.107465,0.190476,0.304698,00:42
3,4.071431,5.600048,0.116474,0.200450,0.318855,00:43
4,3.822079,5.507705,0.120013,0.213964,0.338160,00:44
5,3.607021,5.411248,0.130309,0.227156,0.350064,00:44
6,3.374002,5.318358,0.142214,0.237452,0.373874,00:42
7,3.198809,5.230437,0.145431,0.250643,0.386100,00:43
8,2.957536,5.134071,0.152188,0.259652,0.403475,00:42
9,2.776174,5.037908,0.166667,0.274131,0.424389,00:43


{'train_loss': 0.004325238987803459, 'valid_loss': 4.276210308074951, 'accuracy': 0.3207850754261017, 'top_3_accuracy': 0.45045045018196106, 'top_10_accuracy': 0.5981338620185852, 'tta_accuracy': 0.36068210005760193, 'tta_top_3_accuracy': 0.4993565082550049, 'tta_top_10_accuracy': 0.6463963985443115}


In [8]:
mlflow.set_experiment("bad-rls-top-10-genera")
with mlflow.start_run(run_name="initial"):
    learner = experiments.create_reproducible_learner(
        resnet18,
        Path("/home/yanir/projects/deep-fish/data/rls-top-10-genera"),
        db_kwargs=dict(get_y=lambda path: path.name.split("-")[0].capitalize()),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    learner.fine_tune(120)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

INFO: 'rls-top-10-genera' does not exist. Creating a new experiment


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,time
0,2.944921,1.282301,0.569620,0.818565,00:09


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,time
0,1.628212,1.067808,0.641350,0.869198,00:10
1,1.414558,0.949522,0.683544,0.886076,00:11
2,1.281061,0.872254,0.717300,0.894515,00:10
3,1.150527,0.806776,0.751055,0.907173,00:11
4,1.036941,0.779857,0.759494,0.919831,00:11
5,0.943573,0.734458,0.763713,0.928270,00:11
6,0.838563,0.700131,0.772152,0.932489,00:11
7,0.749407,0.674728,0.776371,0.936709,00:12
8,0.675224,0.650638,0.776371,0.936709,00:12
9,0.617688,0.675598,0.767932,0.940928,00:12


{'train_loss': 0.002694022608920932, 'valid_loss': 0.6393607258796692, 'accuracy': 0.8481012582778931, 'top_3_accuracy': 0.9704641103744507, 'tta_accuracy': 0.9240506291389465, 'tta_top_3_accuracy': 1.0}


## Experiments after the duplication bug fix

In [3]:
mlflow.set_experiment("rls-species-min-images-2")
with mlflow.start_run(run_name="shorter-freeze"):
    class_names = [
        experiments.get_species_from_path(path)
        for path in Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2").ls()
    ]
    print(f"{len(class_names)} images, {len(set(class_names))} classes")
    learner = experiments.create_reproducible_learner(
        resnet18,
        Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2"),
        db_kwargs=dict(splitter=TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(224, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("splitter", "TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)")
    mlflow.log_param("frozen_epochs", 5)
    mlflow.log_param("unfrozen_epochs", 120)
    learner.fine_tune(120, freeze_epochs=5)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

6148 images, 1687 classes


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,9.333320,8.087052,0.000976,0.002928,0.005856,00:31
1,9.057518,7.728109,0.001952,0.007157,0.020820,00:32
2,8.380974,7.100917,0.016916,0.035459,0.074496,00:34
3,7.409082,6.484330,0.045543,0.086532,0.152895,00:36
4,6.353333,5.962341,0.077424,0.141509,0.253090,00:36


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,4.639127,5.906049,0.084906,0.153221,0.271633,00:43
1,4.448617,5.832282,0.091412,0.169486,0.289525,00:43
2,4.211098,5.732248,0.103774,0.185101,0.301887,00:43
3,3.993761,5.641067,0.112232,0.195185,0.315550,00:43
4,3.816918,5.553034,0.118738,0.204945,0.333116,00:43
5,3.612438,5.463369,0.125895,0.216005,0.345803,00:43
6,3.391628,5.364462,0.138907,0.229018,0.358816,00:44
7,3.169010,5.277137,0.144763,0.244958,0.374756,00:43
8,2.956870,5.184620,0.147040,0.252115,0.390371,00:42
9,2.728267,5.123848,0.153221,0.267729,0.401757,00:42


{'train_loss': 0.004129873588681221, 'valid_loss': 4.314911842346191, 'accuracy': 0.32042941451072693, 'top_3_accuracy': 0.4603123068809509, 'top_10_accuracy': 0.5946649312973022, 'tta_accuracy': 0.3552374839782715, 'tta_top_3_accuracy': 0.4954456686973572, 'tta_top_10_accuracy': 0.6353285908699036}


In [3]:
mlflow.set_experiment("rls-species-min-images-2")
with mlflow.start_run(run_name="crop-448-min-scale-0.5-longer-training"):
    class_names = [
        experiments.get_species_from_path(path)
        for path in Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2").ls()
    ]
    print(f"{len(class_names)} images, {len(set(class_names))} classes")
    learner = experiments.create_reproducible_learner(
        resnet18,
        Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2"),
        db_kwargs=dict(
            splitter=TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19),
            item_tfms=RandomResizedCrop(448, min_scale=0.5),
        ),
        dls_kwargs=dict(bs=16),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(448, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms()")
    mlflow.log_param("bs", 16)
    mlflow.log_param("splitter", "TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)")
    mlflow.log_param("frozen_epochs", 10)
    mlflow.log_param("unfrozen_epochs", 200)
    learner.fine_tune(200, freeze_epochs=10)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

6148 images, 1687 classes


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,9.239976,8.049796,0.000325,0.001627,0.007807,02:05
1,8.957006,7.813291,0.000976,0.004880,0.013663,02:26
2,8.393040,7.424610,0.004229,0.012362,0.033832,02:25
3,7.658385,6.984257,0.023097,0.042615,0.082628,02:25
4,6.989848,6.571150,0.036435,0.074821,0.146389,02:25
5,6.267218,6.209653,0.058230,0.121991,0.217632,02:25
6,5.703951,5.885993,0.086207,0.160377,0.269356,02:25
7,5.064371,5.616416,0.104099,0.180872,0.311321,02:25
8,4.520766,5.362996,0.129798,0.218282,0.358165,02:25
9,3.950106,5.145180,0.148341,0.255368,0.395250,02:25


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,2.820222,5.126384,0.158426,0.271958,0.408588,02:56
1,2.576782,5.047933,0.173064,0.283344,0.426480,02:56
2,2.439950,4.979662,0.179896,0.293103,0.440143,02:56
3,2.303966,4.903941,0.190956,0.305791,0.452830,02:56
4,2.104176,4.829595,0.201692,0.318478,0.461288,02:56
5,1.908389,4.785846,0.210150,0.331165,0.471047,02:56
6,1.819934,4.718122,0.220885,0.345478,0.487638,02:56
7,1.670756,4.660517,0.215029,0.348081,0.497072,02:56
8,1.582148,4.621787,0.226090,0.351334,0.500976,02:56
9,1.472748,4.576589,0.229668,0.362720,0.510085,02:57


{'train_loss': 0.0071147289127111435, 'valid_loss': 3.4596235752105713, 'accuracy': 0.43656474351882935, 'top_3_accuracy': 0.576772928237915, 'top_10_accuracy': 0.6935588717460632, 'tta_accuracy': 0.46486663818359375, 'tta_top_3_accuracy': 0.6151593923568726, 'tta_top_10_accuracy': 0.7280416488647461}


In [4]:
mlflow.set_experiment("rls-species-min-images-2")
with mlflow.start_run(run_name="crop-448-min-scale-0.5-longer-training-aug-mult-2.0"):
    class_names = [
        experiments.get_species_from_path(path)
        for path in Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2").ls()
    ]
    print(f"{len(class_names)} images, {len(set(class_names))} classes")
    learner = experiments.create_reproducible_learner(
        resnet18,
        Path("/home/yanir/projects/deep-fish/data/rls-species-min-images-2"),
        db_kwargs=dict(
            splitter=TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19),
            item_tfms=RandomResizedCrop(448, min_scale=0.5),
            batch_tfms=aug_transforms(mult=2.0),
        ),
        dls_kwargs=dict(bs=16),
    )
    mlflow.log_param("model", learner.arch.__name__)
    mlflow.log_param("item_tfms", "RandomResizedCrop(448, min_scale=0.5)")
    mlflow.log_param("batch_tfms", "aug_transforms(mult=2.0)")
    mlflow.log_param("bs", 16)
    mlflow.log_param("splitter", "TrainTestSplitter(test_size=0.5, stratify=class_names, random_state=19)")
    mlflow.log_param("frozen_epochs", 10)
    mlflow.log_param("unfrozen_epochs", 200)
    learner.fine_tune(200, freeze_epochs=10)
    metrics = experiments.get_learner_metrics_with_tta(learner, tta_prefix="tta_", beta=0)
    print(metrics)
    mlflow.log_metrics(metrics)

6148 images, 1687 classes


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,9.262212,8.022528,0.000651,0.003253,0.007157,02:05
1,9.002370,7.805812,0.001952,0.004229,0.012362,02:24
2,8.520694,7.442733,0.003578,0.012362,0.025699,02:25
3,7.904396,7.012458,0.016265,0.037411,0.078399,02:25
4,7.240744,6.643499,0.035459,0.068315,0.141184,02:26
5,6.667822,6.284785,0.052700,0.111256,0.200390,02:26
6,6.187009,5.969697,0.080026,0.146064,0.258295,02:26
7,5.668004,5.676250,0.098243,0.179896,0.294079,02:25
8,5.242146,5.398162,0.122316,0.212101,0.354912,02:25
9,4.725309,5.189806,0.142485,0.255693,0.388744,02:25


epoch,train_loss,valid_loss,accuracy,top_3_accuracy,top_10_accuracy,time
0,3.734060,5.172822,0.167534,0.267729,0.408263,02:56
1,3.549841,5.097252,0.174040,0.283344,0.421926,02:56
2,3.388622,4.985492,0.184776,0.300586,0.436890,02:57
3,3.242408,4.916243,0.195185,0.311321,0.453155,02:56
4,2.991029,4.831339,0.201692,0.325309,0.470072,02:56
5,2.888459,4.771821,0.210800,0.329213,0.474301,02:57
6,2.716056,4.706367,0.218933,0.342550,0.494470,02:57
7,2.602612,4.642037,0.223162,0.349057,0.504554,02:56
8,2.528199,4.554227,0.239102,0.362720,0.518868,02:56
9,2.367545,4.522517,0.239102,0.370202,0.516916,02:56


{'train_loss': 0.044783275574445724, 'valid_loss': 3.3821768760681152, 'accuracy': 0.4616135358810425, 'top_3_accuracy': 0.6050748229026794, 'top_10_accuracy': 0.7225114107131958, 'tta_accuracy': 0.46974626183509827, 'tta_top_3_accuracy': 0.6226415038108826, 'tta_top_10_accuracy': 0.7410539984703064}
